### Assignment: Modelling and Simulations 
*Allan G. Schweinfurth*

We start with a preamble, calling the required packages and commands for a functional code.

In [ ]:

from pyprojroot import here
workspace_path = str(here())
%cd $workspace_path
print(f"Working Directory has been set to `{workspace_path}`")

from os import path
import autolens as al
import autolens.plot as aplt
import autofit as af

### 1. Grid Setup

The first thing we have to do is generate a `Grid2D` object. This is where the ray tracing will be taking place. The `shape_native` value gives the dimension of the grid and the value of `pixel_scales` is the arcsecond-to-pixel conversion factor. 
I.e, for a 250x250 grid and a conversion factor of 0.02", this yields a 5"x5" grid.

In [ ]:
grid_2d  = al.Grid2D.uniform(shape_native=(1000, 1000), pixel_scales=0.02)

We can see this grid with:

In [ ]:
mat_plot_2d = aplt.MatPlot2D(
    title=aplt.Title(label="Grid2D")
)

grid_2d_plotter = aplt.Grid2DPlotter(grid=grid_2d, mat_plot_2d=mat_plot_2d)
grid_2d_plotter.figure_2d()

### 2. Galaxies

Next up, we would like to generate the galaxies, which are `al.Galaxy` objects. There's two important distinctions to be made. The `lens_galaxy` and the `source_galaxy`. 
* the `lens_galaxy` is the galaxy closer (lower redshift) to the observer. Its gravity acts as a *lens* and thus the name
* the `source_galaxy` is the galaxy whose light path was distorted by the `lens_galaxy`. It is farther away and has a higher redshift.

All galaxies have certain attributes depending on their morphology, luminosity, mass, etc. These are defined during the object initialisation.

#### 2.1 Lens Galaxy

Let's create our `lens_galaxy` first. We know this galaxy needs to have a mass distribution, and a light distribution. 

In [ ]:

lens_galaxy = al.Galaxy(
    redshift=0.5,
    mass=al.mp.Isothermal(
        centre=(0,0), 
        ell_comps=(0.0,0.0), 
        einstein_radius=5
    ),
    disk=al.lp.ExponentialSph(
        centre=(0,0),
        effective_radius=1,
        intensity=0.75
    ),
    bulge=al.lp.ExponentialCoreSph(
        centre=(0,0),
        effective_radius=0.25,
        intensity=1
    )
)

We can see what our `lens_galaxy` look like. For that we define a `galaxy_plotter` object with the galaxy and grid we would like to plot, followed by calling the `.figures_2d(image=True)` method to generate the image. We also use an `include2d` object to hide unnecessary information (caustics, critical curves, etc.)

In [ ]:
include_2d = aplt.Include2D(
    origin=False, 
    mask=False, 
    border=False, 
    light_profile_centres=False, 
    tangential_critical_curves=False,
    radial_critical_curves=False,
    mass_profile_centres=False,
    radial_caustics=False,
    tangential_caustics=False
)
galaxy_plotter = aplt.GalaxyPlotter(galaxy=lens_galaxy,grid=grid_2d, include_2d=include_2d) 
galaxy_plotter.set_title('Lens Galaxy')
galaxy_plotter.figures_2d(image=True)


Even more, we can see each light profile individually (that of the `Disk` and that of the `Bulge`)

In [ ]:
galaxy_plotter.subplot_of_light_profiles(image=True)



#### 2.2 Source Galaxy

Now our `source_galaxy`. Wether or not we define a mass for this galaxy is irrelevant, as only its light plays a role in the lensing process. 

In [ ]:
source_galaxy = al.Galaxy(
    redshift=2.5,
    bulge=al.lp.DevVaucouleursSph(
        centre=(-1, 0), 
        intensity=0.01, 
        effective_radius=0.2
    ),
    disk=al.lp.Exponential(
        centre=(-1, 0),
        ell_comps=(0, 0.5),
        intensity=1,
        effective_radius=0.7
    ),
)

Similarly, we can plot this galaxy by changing the galaxy in the `galaxy_plotter` object

In [ ]:
galaxy_plotter = aplt.GalaxyPlotter(galaxy=source_galaxy,grid=grid_2d, include_2d=include_2d) 
galaxy_plotter.set_title('Source Galaxy')
galaxy_plotter.figures_2d(image=True)
galaxy_plotter.subplot_of_light_profiles(image=True)


A very nice disk galaxy!

### 3. Tracer


The `tracer` is the object that performs the ray tracing. It works with `planes`, where each plane contains a group of galaxies at the same redshift. It also includes a `cosmology` which is used to compute how space gets distorted. We'll be using the `FlatLambdaCDM` model with $H_0=70 km\cdot s^{-1}/kpc$ and $\Omega_0=0.3$

In [ ]:
tracer=al.Tracer.from_galaxies(
    galaxies=[lens_galaxy,source_galaxy],
    cosmology=al.cosmo.FlatLambdaCDM(70,0.3)
)

Now, we can see the results of the tracing. Similar to the Galaxies we use a `.tarcerPlotter` method to generate a plot

In [ ]:
tracer_plotter = aplt.TracerPlotter(tracer=tracer,grid=grid_2d, include_2d=include_2d)
tracer_plotter.figures_2d(image=True) 

We can see each individual plane, using the `figures_2d_of_planes(plane_index=i,plane_image=True)` method, where $i$ is 0 for the lens plane and 1 for the source plane

In [ ]:
tracer_plotter.figures_2d_of_planes(plane_index=0, plane_image=True)
tracer_plotter.figures_2d_of_planes(plane_index=1, plane_image=True)


Moreover, we can see several sublots of important data (convergence, potential, deflection angles) using 

In [ ]:
# tracer_plotter.subplot_tracer()

### 4. Observatory View

We now want to know how this will look through an actual telescope, to make it more realistic with real observations.

#### 4.1 Hubble Space Telescope

We first define the Point Spread Function (PSF), which is a characteristic value of any optical system. The PSF is then convoluted with the image. For the Hubble Space Telescope. (Taken from the package documentation)

In [ ]:
psf_2d = al.Kernel2D.from_gaussian(
    shape_native=(11, 11), sigma=0.1, pixel_scales=grid_2d.pixel_scales
)

Now we "pad" the image, so the edge effects don't affect the convolution with the PSF

In [ ]:
normal_image_2d = tracer.image_2d_from(grid=grid_2d)
padded_image_2d = tracer.padded_image_2d_from(
    grid=grid_2d, psf_shape_2d=psf_2d.shape_native
)

Now that the image has been prepared, we can begin by setting up a `simulator` in which we specify `exposure_time`,`psf`, `background_sky_level` and `poisson_noise`

In [ ]:
simulator = al.SimulatorImaging(
    exposure_time=300.0, psf=psf_2d, background_sky_level=0.1, add_poisson_noise=True
)

Now we can plot this, similar to how it was plotted with `tracer` and `galaxy`

In [ ]:
imaging = simulator.via_tracer_from(tracer=tracer, grid=grid_2d)
imaging_plotter = aplt.ImagingPlotter(imaging=imaging)
imaging_plotter.figures_2d(image=True)


in addition, the `,subplot_imaging()` method provides some valuable data on the "observation", including the noise-map and the chi squared map

In [ ]:
imaging_plotter.subplot_imaging()

We can output all this data to a `.fits` files, which is the standard in astronomy.

In [ ]:
dataset_path = path.join("dataset", "imaging", "dataforfit")
imaging.output_to_fits(
    image_path=path.join(dataset_path, "image.fits"),
    noise_map_path=path.join(dataset_path, "noise_map.fits"),
    psf_path=path.join(dataset_path, "psf.fits"),
    overwrite=True,
)

#### 5. Modelling the System


First we import the results from the previous step (this is not necessary but in real life the data would need to be imported)

In [ ]:
imaging = al.Imaging.from_fits(
    image_path=path.join(dataset_path, "image.fits"),
    noise_map_path=path.join(dataset_path, "noise_map.fits"),
    psf_path=path.join(dataset_path, "psf.fits"),
    pixel_scales=0.02,
)

Next up we'll prepare the data. We first apply a mask to it, such that the background noise from empty space poses no problem in the fitting.

In [ ]:
imaging_plotter = aplt.ImagingPlotter(imaging=imaging)
imaging_plotter.subplot_imaging()

mask_2d = al.Mask2D.circular(
    shape_native=imaging.shape_native, pixel_scales=imaging.pixel_scales, radius=8
)

imaging = imaging.apply_mask(mask=mask_2d)

imaging_plotter = aplt.ImagingPlotter(imaging=imaging)
imaging_plotter.subplot_imaging()




Now we create a model with which the search will be performed. From observation, the redshift can be estimated as well as the light and mass distributions. In real life, several different light and mass profiles would be tested as sometimes the imaging doesn`t exactly provide much information on this, for instance when the lensing is so strong the image is totally distorted. This applies usually to the source galaxy, as the lens galaxy is directly obsevable and its properties can be determined. IN addition to this we can limit some of the priors and assume a probability distribution for certain attributes such as centre, radius, intensity, etc.

Initial model of the lens galaxy:

In [ ]:
mass_lns = af.Model(al.mp.Isothermal)
bulge_lns = af.Model(al.lp.Sersic)
disk_lns = af.Model(al.lp.ExponentialSph)

mass_lns.centre.centre_0 = af.UniformPrior(lower_limit=-0.05, upper_limit=0.05)
mass_lns.centre.centre_1 = af.UniformPrior(lower_limit=-0.05, upper_limit=0.05)
bulge_lns.centre.centre_0 = af.UniformPrior(lower_limit=-0.05, upper_limit=0.05)
bulge_lns.centre.centre_1 = af.UniformPrior(lower_limit=-0.05, upper_limit=0.05)
disk_lns.centre.centre_0 = af.UniformPrior(lower_limit=-0.05, upper_limit=0.05)
disk_lns.centre.centre_1 = af.UniformPrior(lower_limit=-0.05, upper_limit=0.05)
mass_lns.ell_comps.ell_comps_0 = af.GaussianPrior(
    mean=0.0, sigma=0.3, lower_limit=-0.1, upper_limit=0.1
)
mass_lns.ell_comps.ell_comps_1 = af.GaussianPrior(
    mean=0.0, sigma=0.3, lower_limit=-0.1, upper_limit=0.1
)
disk_lns.effective_radius = af.GaussianPrior(
    mean=1.0, sigma=0.8, lower_limit=0.0, upper_limit=2
)
bulge_lns.effective_radius = af.GaussianPrior(
    mean=1.0, sigma=0.8, lower_limit=0.0, upper_limit=2
)
mass_lns.einstein_radius = af.GaussianPrior(
    mean=5, sigma=0.2, lower_limit=2.5, upper_limit=7.5
)
lens_galaxy_model = af.Model(al.Galaxy, redshift=0.5, mass=mass_lns, disk=disk_lns, bulge=bulge_lns)


similarly for the source galaxy

In [ ]:
bulge_src=af.Model(al.lp.DevVaucouleursSph)
disk_src=af.Model(al.lp.Exponential)


disk_src.effective_radius = af.UniformPrior(lower_limit=0.0, upper_limit=1)
bulge_src.effective_radius = af.UniformPrior(lower_limit=0.0, upper_limit=1)

source_galaxy_model = af.Model(al.Galaxy, redshift=2.5,bulge=bulge_src, disk=disk_src)


And lastly we create our model system

In [ ]:
model = af.Collection(galaxies=af.Collection(lens=lens_galaxy_model, source=source_galaxy_model)) 

Now we can start the search, for this we'll use the `Dynesty` tool from `autofit`, which performs a non-linear search from the imaging. This part of the code is very cpu-intensive, thus it should be ran as a code rather than a jupyter notebook.

In [ ]:

search = af.DynestyStatic(
    path_prefix=path.join(dataset_path,"Assignment"),
    name="search",
    unique_tag="01",
    nlive=40,
    walks=5,
    force_x1_cpu=False
)

analysis = al.AnalysisImaging(dataset=imaging)

print(
    "Dynesty is running")

result = search.fit(model=model, analysis=analysis)

print("Results Info:")
print(result.info)

fit_imaging_plotter = aplt.FitImagingPlotter(fit=result.max_log_likelihood_fit)
fit_imaging_plotter.subplot_fit_imaging()